In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import time
import re

In [95]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.options.display.float_format = '{:.2f}'.format

In [120]:
def get_comments(service, df, contentNo = 52001198):
    TIMEOUT = 1   
    driver = webdriver.Chrome(service=service)
    
    base_url = 'https://page.kakao.com/content/'
    episode_url = base_url + str(contentNo)   
        
    print(episode_url)
    driver.get(episode_url)
    driver.implicitly_wait(TIMEOUT)
    
    # 전체 댓글 클릭
    driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[2]/div/div/div[1]/div/div[2]/div').click()
    time.sleep(1)
    
    # 댓글 (149,394)
    num_reviews = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[2]/div[1]/div[2]/div[2]/div[2]/div/div/div[1]/span')
    num_reviews = int(re.sub(r'[^0-9]', '', num_reviews.text))
    print("num_reviews: " + str(num_reviews))
    
    # 크롤링할 전체 페이지수, 149,394 / 25 = 5975.76 pages
    reviews_per_page = 25
    num_pages  = num_reviews / reviews_per_page + 1
    print("num_pages: " + str(int(num_pages)))
    print("Start crawling.") 
          
    cnt_page = 0
    for i in range(2): #range(int(num_pages)):
        for page in range(10) : # 25 reviews per page , 1 ~ 10 pages

            # Extract dates
            dates = driver.find_elements(By.CSS_SELECTOR,'div.css-jkb0z2-Comment > div.css-1ucu4d6-Comment > span.css-1ur8mkx-Text-Comment')
            dates = [date.text for date in dates]        

            # Extract reveiws
            reviews = driver.find_elements(By.CSS_SELECTOR,'div.css-jkb0z2-Comment > span.css-odpwuz-Text-Comment')
            reviews = [review.text for review in reviews]

            # Extract episodes
            episodes = driver.find_elements(By.CSS_SELECTOR,'div.css-1lm99l8-Comment > div.css-jkb0z2-Comment > span.css-1j3h6tm-Text-Comment')
            episodes = [episode.text for episode in episodes]     

            df_page = pd.DataFrame(data=[], columns=['contentNo','Date','Review','Episode'])       
            df_page['Date'] = dates
            df_page['Review'] = reviews
            df_page['Episode'] = episodes
            df_page['contentNo'] = df_page['contentNo'].fillna(int(contentNo))
            df = pd.concat([df,df_page]) 

            cnt_page += 1
            current_page = driver.find_element(By.CSS_SELECTOR, 'span.css-jwqqli-Text-Pagination')
            print(current_page.text,' page done.')

            # 크롤링한 페이지수가 크롤링해야 할 전체 페이지 수와 같아지면 종료
            if cnt_page == int(num_pages):
                return df            
            
            # 아니라면 계속 크롤링
            if page != 9: # 마지막 페이지가 아니라면 Move to the next page  (current page 의 next sibling)
                driver.find_element(By.CSS_SELECTOR, 'span.css-jwqqli-Text-Pagination+span').click()        
                time.sleep(0.2)
            else: 
                print("Move to the next page.") # 마지막 페이지(10단위의 페이지)라면 Move to the next 10 page( click > )
                driver.find_element(By.CSS_SELECTOR, 'div.css-pv33ym-Pagination > div.css-t0ig9f-Pagination').click()        
                time.sleep(0.2) 
    return df

# create dataframe
df = pd.DataFrame(data=[], columns=['contentNo','Date','Review','Episode'])

# 전지적 독자 시점 - ebook

# contentNo setting
contentNo = 52001198

service = Service(executable_path='c:\Temp\chromedriver.exe')

# get all comments
df = get_comments(service, df, contentNo)

https://page.kakao.com/content/52001198
num_reviews: 149395
num_pages: 5976
Start crawling.
1  page done.
2  page done.
3  page done.
4  page done.
5  page done.
6  page done.
7  page done.
8  page done.
9  page done.
10  page done.
Move to the next page.
11  page done.
12  page done.
13  page done.
14  page done.
15  page done.
16  page done.
17  page done.
18  page done.
19  page done.
20  page done.
Move to the next page.


In [123]:
#Save dataframe
DATA_PATH = './data/'
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
file_path = './data/omniscient-reader_Ebook_KaKaoPage_{contentNo}.csv'.format(contentNo=contentNo)
df.to_csv(file_path,index = False)
print("Save Done.")
df

Save Done.


,contentNo,Date,Review,Episode
0,52001198,2018.11.10,내가 성격이 좀 못되먹어서 가끔좀 재수없게 행동할때가있어. 내가 꺼지라고하면 잠시 ...,전지적 독자 시점 256화
1,52001198,2018.11.08,뷰어 이상한거 쓰더니 판매중지되고 뷰어 바뀐게 훨씬 낫네요. 문피아 2018분기 1...,전지적 독자 시점 1화
2,52001198,2018.11.09,존나 시끄럽게 울리고 있다는게 제일 웃겨ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㄱㅋ태연하게 무시하고 유중...,전지적 독자 시점 142화
3,52001198,2018.11.09,흑염룡 대사랑 그녀석 어릴땐 참 작았는데...하는말이 이어져있으니까 이상하게 들리잖...,전지적 독자 시점 173화
4,52001198,2018.11.08,그 진짜 오지긴 한데 저 모든걸 거적데기 걸치고 덜렁거리는 흑염룡과 함께 했다고 생...,전지적 독자 시점 56화
5,52001198,2018.11.08,존나 웃겨 시발 이새끼ㅋㅋㅋㄱㄱㄱㄱㄹㄹㄹㄹㅋㄷㄷㄷㅋㄷㄴㅌㄱㄷㄷㄱㄱㅋㅋㅋ개사기잖아ㅋㄱ...,전지적 독자 시점 38화
6,52001198,2018.12.31,김독자. 에반데\n유중혁. 에반데\n비유. 에바앗\n삼진 에바로 수락되었습니다,전지적 독자 시점 335화
7,52001198,2018.11.09,나 사실 정말 독자랑 중혁이랑 이어졌으면 좋겠어,전지적 독자 시점 151화
8,52001198,2018.11.09,ㅋㅋㄱㅋㅋㅋㅋㄱㅋㅋㅋㅋㅋㅋㅋㅋ진짜 맨날 서로 이름 팔고있어ㅋㅋㅋㅋㅋㅋ,전지적 독자 시점 196화
9,52001198,2018.11.09,대박.... 저렇게까지 너랑 안헤어지겠다고 발버둥치는데 서비스로 대환단은 입으로 먹여줘라,전지적 독자 시점 269화
